In [29]:
from datasets import Dataset
from time import perf_counter
from autofaiss import build_index
from tqdm import tqdm
import polars as pl
import numpy as np
from core_pro.ultilities import make_sync_folder

In [30]:
path = make_sync_folder('dataset/item_matching')
file = path / 'data_sample_FMCG_clean.parquet'

df = pl.read_parquet(file)
df.head()

index,item_id,item_name,shop_id,shop_name,level1_global_be_category,level2_global_be_category,level3_global_be_category,cluster,description,images,image_url,item_name_clean,file_path
u32,i64,str,i64,str,str,str,str,str,str,str,str,str,str
0,19092271907,"""LINEABON K2D3 nhập khẩu châu â…",1053944724,"""ChiChi.Kids17""","""Mom & Baby""","""Baby Healthcare""","""Baby Vitamins & Supplements""","""FMCG""","""Nguồn gốc xuất xứ K2 D3 Lineab…","""vn-11134207-7ras8-m49p5xv9ecb3…","""http://f.shopee.vn/file/vn-111…","""lineabon k2d3 nhập khẩu châu â…","""/media/kevin/data_4t/dataset/i…"
1,17398587723,"""[MKB Gift] Bộ 3 Khăn ướt Moony…",63522286,"""Moony - Gian Hàng Chính Hãng""","""Mom & Baby""","""Bath & Body Care""","""Wipes""","""FMCG""","""[{""t"":""Bộ 3 khăn ướt Moony 50 …","""eb26f55f7359e0de333f0c34e2619f…","""http://f.shopee.vn/file/eb26f5…","""bộ 3 khăn ướt moony 50 miếng/g…","""/media/kevin/data_4t/dataset/i…"
2,19111299596,"""Bàn Chải Đánh Răng Lipzo Sensi…",170502615,"""Niva Lipzo Official Store""","""Health""","""Personal Care""","""Oral Care""","""FMCG""","""[{""t"":""BÀN CHẢI ĐÁNH RĂNG LIPZ…","""vn-11134207-7ras8-m2kwpjg8vb46…","""http://f.shopee.vn/file/vn-111…","""bàn chải đánh răng lipzo sensi…","""/media/kevin/data_4t/dataset/i…"
3,25583412760,"""Rong biển ăn liền Bibizan siêu…",119247917,"""King 21""","""Food & Beverages""","""Snacks""","""Seaweed""","""FMCG""","""NGÀY IN TRÊN BAO BÌ LÀ NGÀY SẢ…","""vn-11134207-7r98o-lxqfhxwp3h0b…","""http://f.shopee.vn/file/vn-111…","""rong biển ăn liền bibizan siêu…","""/media/kevin/data_4t/dataset/i…"
4,15930150764,"""(MUA 2 SON TẶNG 1 TÚI, 1 CUSHI…",181790483,"""Lam Thảo Cosmetics""","""Beauty""","""Makeup""","""Lips""","""FMCG""","""Son Tint Bóng Espoir Couture L…","""vn-11134207-7r98o-lyrll7tp71y9…","""http://f.shopee.vn/file/vn-111…","""son tint bóng espoir couture l…","""/media/kevin/data_4t/dataset/i…"


In [66]:
path_array = path / 'tmp/array/bge/embed.npy'
vectors = np.load(path_array)
items = df[['item_id', 'item_name']].to_numpy().tolist()

In [88]:
from vicinity import Vicinity, Backend, Metric

vicinity = Vicinity.from_vectors_and_items(
    vectors=vectors,
    items=items,
    backend_type=Backend.FAISS,
    metric=Metric.COSINE
)

In [89]:
results = vicinity.query(vectors[:5], k=5)

In [90]:
(
    pl.DataFrame()
    .with_columns(
        pl.Series('match', [[i[0][0] for i in _] for _ in results]),
        pl.Series('item_name', [[i[0][-1] for i in _] for _ in results]),
        pl.Series('score', [[i[-1] for i in _] for _ in results])
    )
    .explode(['match', 'item_name', 'score'])
)

match,item_name,score
i64,str,f32
19092271907,"""LINEABON K2D3 nhập khẩu châu â…",1.000008
7820117814,"""D3 K2 Lineabon bổ sung vitamin…",0.816858
29755293447,"""D3 K2 Lineabon bổ sung Vitamin…",0.800006
29561676142,"""[Mẫu mới 15ml] LineaBon vitami…",0.799012
16154798905,"""Vitamin lineabon k2 d3 bổ sung…",0.790578
…,…,…
15930150764,"""(MUA 2 SON TẶNG 1 TÚI, 1 CUSHI…",0.999896
16126980507,"""SON TINT BÓNG ESPOIR COUTURE L…",0.987997
15369528293,"""Son Kem Espoir Couture Lip Tin…",0.863823
